In [2]:
# fuck this

import numpy as np

with open("input.txt") as f:
    text = f.readlines()

boxes = [[int(num) for num in line[:-1].split(",")] for line in text]
boxes = np.array(boxes)
boxes_x, boxes_y, boxes_z = boxes.T # works 👍

length = boxes.shape[0] # number of rows, of boxes

x_matrix = np.full((length, length), boxes_x) #
y_matrix = np.full((length, length), boxes_y) # try to parallelize all of these
z_matrix = np.full((length, length), boxes_z) #
distance = np.sqrt(np.square(x_matrix - x_matrix.T) + np.square(y_matrix - y_matrix.T) + np.square(z_matrix - z_matrix.T))
distance

array([[     0.        ,  49774.83627296,  90706.3554609 , ...,
         84775.6524363 ,  51604.08966351,  20133.40857878],
       [ 49774.83627296,      0.        ,  47594.10199804, ...,
         43882.99764146,  61231.12553269,  45518.92679974],
       [ 90706.3554609 ,  47594.10199804,      0.        , ...,
         13340.83689279, 107928.30363255,  80531.42993888],
       ...,
       [ 84775.6524363 ,  43882.99764146,  13340.83689279, ...,
             0.        , 104936.36528868,  76725.14553261],
       [ 51604.08966351,  61231.12553269, 107928.30363255, ...,
        104936.36528868,      0.        ,  60754.36116856],
       [ 20133.40857878,  45518.92679974,  80531.42993888, ...,
         76725.14553261,  60754.36116856,      0.        ]],
      shape=(1000, 1000))

In [14]:
from bisect import insort

# my personal sort, O(n^2)
min_1000 = []
last = np.inf
for i, row in enumerate(distance):
    for j, element in enumerate(row):
        if i == j: break # ignore the upper triangle of the matrix, it's symmetric
        if element < last:
            insort(min_1000, (element, i, j))
            if len(min_1000) > 1000:
                min_1000.pop()
            last = min_1000[-1][0]

min_1000

[(np.float64(984.2936553691687), 658, 38),
 (np.float64(1062.2447928796828), 921, 310),
 (np.float64(1181.7639358179788), 881, 651),
 (np.float64(1209.9735534299914), 774, 550),
 (np.float64(1286.1531790576114), 583, 415),
 (np.float64(1308.875089532993), 419, 350),
 (np.float64(1662.1663574985507), 264, 159),
 (np.float64(1663.6279632177382), 741, 213),
 (np.float64(1724.764911516929), 769, 685),
 (np.float64(1821.2287061212273), 659, 534),
 (np.float64(1822.200043902974), 764, 534),
 (np.float64(1822.9111333249352), 764, 659),
 (np.float64(1922.736331377758), 975, 220),
 (np.float64(2076.4089192642186), 235, 119),
 (np.float64(2095.247240780906), 980, 268),
 (np.float64(2109.629825348514), 795, 722),
 (np.float64(2116.9945677776313), 979, 896),
 (np.float64(2117.777372624422), 447, 364),
 (np.float64(2166.5458684274377), 548, 87),
 (np.float64(2178.2612331857717), 379, 206),
 (np.float64(2200.412688565488), 955, 506),
 (np.float64(2268.1732738042742), 477, 231),
 (np.float64(2300.396

In [15]:
# assigning circuits, O(n^2)
assigned_circuit = [None] * 1000
last = 1
for i in range(1000):
    _, start, end = min_1000[i]
    assigned_start = assigned_circuit[start]
    assigned_end = assigned_circuit[end]
    if assigned_start is None and assigned_end is None:
        circuit = last
    elif assigned_start is None:
        circuit = assigned_end
    elif assigned_end is None:
        circuit = assigned_start
    else: # both already part of different circuits
        circuit = assigned_start
        for j in range(1000):
            if assigned_circuit[j] == assigned_end:
                assigned_circuit[j] = circuit
    assigned_circuit[start] = circuit
    assigned_circuit[end] = circuit
    last += 1
print(assigned_circuit)

[983, 598, 30, 76, 98, 42, 30, 80, 29, 122, 72, 29, 80, 657, 357, 342, 29, None, 937, 868, None, 357, 582, 17, None, 247, None, 68, 201, 80, 355, 28, None, 234, 135, 663, None, 868, 1, 42, 598, 100, None, 17, 357, 188, 130, 598, 80, 308, 17, 43, 94, 29, 209, 868, 80, 94, 377, 240, 54, 80, 868, 158, 29, 582, 526, 237, 58, 467, None, 260, None, 80, 415, 59, None, 441, 591, 582, 135, 80, 80, 879, 1, 85, 30, 222, 377, 582, 388, None, 199, 335, None, 29, 30, 598, 59, None, 29, 43, 42, None, None, 598, None, 103, 582, 260, 201, 76, 80, 84, None, 29, None, 652, 429, 575, 868, 582, 40, 1, 712, 122, None, 81, None, None, 29, 88, 247, 46, 467, 9, 352, 156, 53, None, 156, 50, 582, 76, 85, 891, 582, 32, 85, 80, 94, 53, 311, 54, 122, 188, 240, None, 80, 582, 29, 103, 598, 100, 473, 598, 208, 115, 76, 4, 85, 28, 80, 582, 309, 46, 94, 375, None, 62, 17, 46, 167, 156, 868, 260, 59, 714, 199, 582, 59, None, 582, 85, 276, 221, 122, 59, 712, 4, 17, 85, None, 598, 167, None, 598, 598, 221, 67, 473, 128, 3

In [21]:
from collections import defaultdict

# making it a dict, O(n)
circuits_dict = defaultdict(set)
for box, circuit in enumerate(assigned_circuit):
    if circuit is None:
        circuit = "None"
    circuits_dict[circuit].add(box)

for i in circuits_dict.items(): print(i)

(983, {0, 666})
(598, {1, 393, 523, 396, 280, 795, 162, 165, 40, 47, 944, 562, 952, 958, 453, 203, 206, 207, 591, 722, 723, 597, 343, 97, 225, 226, 739, 870, 105, 489, 500, 756, 504, 634, 379, 895})
(30, {96, 2, 991, 868, 997, 6, 486, 872, 966, 956, 969, 653, 879, 86, 506, 955, 922, 607})
(76, {962, 3, 168, 844, 622, 143, 111, 304, 654, 243, 816, 911, 916, 858, 603, 348, 351})
(98, {4, 308})
(42, {577, 261, 5, 102, 39, 454, 805, 779, 336, 528, 761, 762})
(80, {643, 7, 12, 655, 784, 149, 405, 533, 408, 789, 282, 792, 668, 285, 29, 158, 286, 418, 674, 809, 298, 937, 172, 559, 48, 305, 688, 818, 436, 946, 439, 56, 823, 827, 61, 322, 965, 327, 712, 73, 839, 842, 333, 81, 82, 850, 219, 228, 996, 999, 233, 362, 492, 876, 877, 112, 754, 892, 637})
(29, {130, 8, 11, 526, 910, 16, 540, 669, 542, 924, 160, 546, 931, 551, 810, 53, 317, 64, 708, 841, 462, 602, 475, 95, 100, 746, 115, 373, 629})
(122, {320, 834, 196, 869, 9, 717, 214, 599, 154, 315, 125})
(72, {10, 525})
(657, {13, 718, 366, 720, 6

In [35]:
max_3 = []
for circ, bx in circuits_dict.items():
    if circ == "None":
        continue
    dimension = len(bx)
    insort(max_3, dimension)
    if len(max_3) > 3: max_3.pop(0)
print(max_3)

[36, 59, 61]


In [36]:
np.prod(max_3)

np.int64(129564)